[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tensorchiefs/dlwbl_eth25/blob/master/notebooks/03_uk_faces.ipynb)

# Probabilistic Age Regression based on images

In this notebook you will use images of faces to set up regression models that outputs a  conditional gaussian probability distribution $p(y|x)$ for the age of the displayed person.

**Dataset:**
You work with a the [UTKFace dataset](https://susanqq.github.io/UTKFace/).  The dataset consists of over 20,000 face images with annotations of age, gender (and ethnicity). The data is already preprocessed and rescaled (80x80 pixels).


**GPU:**
It is better to use the GPU for this notebook. If you are using colab, you can change the runtime type in the menu: Runtime -> Change runtime type -> Hardware accelerator -> GPU.

**Your Task**
Steps through the notebook, try to understand the code and fill out the cells with (🔧 YOUR TASK)

#### Settings



In [ ]:
EPOCHS = 15   #Change this to smaller number for testing, negative means load the weights from dropbox (only for JAX)

In [ ]:
### Running on Colab
import sys
import time
IN_COLAB = 'google.colab' in sys.modules

print(f"Running on Colab: {IN_COLAB} date {time.asctime()} UTC")


##### Imports and setting the backend

In [ ]:
# %matplotlib inline is fine for Jupyter environments
%matplotlib inline

# General-purpose libraries
import os
import sys
import time
import pickle
import urllib
import numpy as np
import pandas as pd
from tqdm import tqdm
from IPython.display import display, Markdown

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Keras + Backend setup (must be before importing keras)
os.environ["KERAS_BACKEND"] = "torch"
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import (
    Dense, Convolution2D, MaxPooling2D, Flatten,
    Activation, Dropout, Input, Concatenate
)
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from tqdm.keras import TqdmCallback

# PyTorch
import torch

# Image handling
from PIL import Image

# Environment & backend checks
print(f'Keras_version: {keras.__version__}')  # e.g. 3.5.0
print(f'Torch_version: {torch.__version__}')  # e.g. 2.5.1+cu121
print(f'Keras backend: {K.backend()}')

cuda_available = torch.cuda.is_available()
cuda_version = torch.version.cuda if cuda_available else "N/A"
print(f"CUDA Available: {cuda_available}")
print(f"CUDA Version: {cuda_version}")

# Device selection
if cuda_available:
    device = torch.device("cuda")
    print("✅ CUDA is available. Using GPU.")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("✅ MPS (Apple Silicon GPU) is available. Using MPS.")
else:
    device = torch.device("cpu")
    print("❌ No GPU available. Using CPU.")

print(f"Device selected: {device}")


#### Getting the data

In [ ]:
if not os.path.isfile('X_faces.npy'):
    urllib.request.urlretrieve(
    "https://www.dropbox.com/s/5m7nmebpjysqtus/X_faces.npy?dl=1",
    "X_faces.npy")
!ls -lh X_face*

if not os.path.isfile('Y_age.npy'):
    urllib.request.urlretrieve(
    "https://www.dropbox.com/s/flpyvgdqoatdw0g/Y_age.npy?dl=1",
    "Y_age.npy")
!ls -lh Y*

In [ ]:
X=np.load("X_faces.npy")
Y=np.load("Y_age.npy")
print(X.shape)
print(Y.shape)

#### Splitting the data into train, val and test dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=201)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=34)
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

#### Looking at a few image samples

In [ ]:
plt.figure(figsize=(8,8))
for i in range(0,9):
    plt.subplot(3,3,i+1)
    plt.imshow(X_train[i])
    plt.title("Age : "+ str(y_train[i]))
plt.show()

#### Normalize the data

In [ ]:
X_train=X_train/255
X_val=X_val/255
X_test=X_test/255

#### Looking at the age distribution

In [ ]:
plt.figure(figsize=(14,6))

plt.subplot(1,2,1)
sns.histplot(y_train, bins=30, kde=True, color='blue', line_kws={'linewidth': 5.5})
plt.title("Age Distribution in Training Set", fontsize=15)
plt.xlabel("Age", fontsize=12)
plt.ylabel("Density", fontsize=12)

plt.subplot(1,2,2)
sns.histplot(y_val, bins=30, kde=True, color='green')
plt.title("Age Distribution in Validation Set", fontsize=15)
plt.xlabel("Age", fontsize=12)
plt.ylabel("Density", fontsize=12)

plt.tight_layout()
plt.show()

#### 🔧 **Exercise 1:** implement Null Model

Use a null model, which simply predicts a Gaussian describing the marginal age distribution (independent of an individual image).

##### i) Estimation of $\mu$ and $\sigma$

You should get $\hat{\mu} \approx 33.14$ and $\hat{\sigma} \approx 19.81$.

##### ii) Calculate the NLL of the null model on the testset

You should get a NLL of $\approx 4.4$.

Recall the density function of the normal distribution:
$$
p(y) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp\left(-\frac{(y-\mu)^2}{2\sigma^2}\right)
$$

In [ ]:
# @title 🔑 Solution Code { display-mode: "form" }
#### 🎯 Solution ########

# i) NLL
mu_naive = np.mean(y_train)
sd_naive = np.std(y_train)

print(f"mu_naive: {mu_naive}")
print(f"sd_naive: {sd_naive}")

# ii) Calculate NLL on testset
nll_naive = 0
for i in range(len(X_test)):  # Iterate over the entire test set
    # Calculate the log probability using NumPy
    log_prob = -0.5 * np.log(2 * np.pi * sd_naive**2) - (y_test[i] - mu_naive)**2 / (2 * sd_naive**2)
    nll_naive -= log_prob  # Accumulate negative log probability

nll_naive /= len(X_test)  # Average the NLL over the test set

print(f"NLL for naive prediction (NumPy): {nll_naive}")

#### Prepare Data


Note that the reshape in the following cell is extremly important. This makes the shape of y from $(B,)$ to $(B,1)$ - a shape of $(B,)$ would cause strange broadcast errors.

In [ ]:
print(X_train.mean())  # check if centered around approx 0.5
print(X_val.mean())
print(X_val.mean())
print("Shape:", y_test.shape)

In [ ]:
##### NOTE THE RESHAPE IS EXTREMLY IMPORTANT ####
y_train = y_train.reshape(-1,1) # this reshape is important!
y_val = y_val.reshape(-1,1)     # it adds a new dimension
y_test = y_test.reshape(-1,1)   # after the last dim (-1)
print(y_train.shape) #Need to have the last dimension of ,1

### Output of NN to Gaussian Distribution

The output NN has the form (B, 2) with the first dimension. The last two dimensions code the mean and the log(sd) of the Gaussian.

In [ ]:
# Wrapper function to convert model output to a PyTorch Normal distribution
import torch
from torch.distributions import Normal

@staticmethod
def output_to_gaussian_distribution(output):
    mean = output[:, 0:1]
    log_sd = output[:, 1:2]
    scale = torch.exp(log_sd)  # Ensure positive scale
    return torch.distributions.Normal(loc=mean, scale=scale)

def NLL(y_true, output):
  dist = output_to_gaussian_distribution(output)
  return -dist.log_prob(y_true).mean()

## Fit a regression model with flexible variance
In the next cells you will again define and fit a model on the face images. You will use a CNN to model the mu parameter of a gaussian conditional probability distribution, but this time the sigma will not be constant for all inputs. Every iamge will be able to have a different sigma. For the loss we use the NLL.

In [ ]:
kernel_size = (3, 3)
pool_size = (2, 2)
input1 = Input(shape=(80,80,3))
x = Convolution2D(16,kernel_size,padding='same',activation="relu")(input1)
x = Convolution2D(16,kernel_size,padding='same',activation="relu")(x)
x = MaxPooling2D(pool_size=pool_size)(x)

x = Convolution2D(32,kernel_size,padding='same',activation="relu")(x)
x = Convolution2D(32,kernel_size,padding='same',activation="relu")(x)
x = MaxPooling2D(pool_size=pool_size)(x)

x = Convolution2D(32,kernel_size,padding='same',activation="relu")(x)
x = Convolution2D(32,kernel_size,padding='same',activation="relu")(x)
x = MaxPooling2D(pool_size=pool_size)(x)
x = Flatten(name = 'bef_split')(x)

# One HEAD
x = Dense(2*500, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(2*50, activation="relu")(x)
x = Dropout(0.3)(x)
mean_and_sd_layer = Dense(2)(x)

model_flex = Model(inputs=input1, outputs=mean_and_sd_layer)

from keras import ops as K  # Use keras.ops instead of keras.backend

# Custom MAE metric
def custom_mae(y_true, y_pred):
    mean_pred = y_pred[:, 0]  # Extract mean predictions
    return K.mean(K.abs(y_true - mean_pred))  # Use ops for math operations

model_flex.compile(optimizer="adam", loss=NLL, metrics=[custom_mae, 'mse'])

In [ ]:
history=model_flex.fit(X_train, y_train,
                      epochs=EPOCHS,
                      verbose=0,
                      validation_data=(X_val, y_val),
                      callbacks=[TqdmCallback(verbose=1)]
                    )
plt.plot(history.history['loss'], label = 'Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.axhline(y=nll_naive, color='r', linestyle='--', label='Null Model NLL')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.ylim(0, 7)
plt.legend()
plt.show()

In [ ]:
#Sort the list and find the 5 highest values
indices =  [0,1,2,3,5]
ages = np.arange(-1,100,0.5)
for i in indices:
    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)
    plt.imshow(X_test[i])
    d = output_to_gaussian_distribution(model_flex(X_test[i:i+1]))
    mean_age = d.mean.cpu().detach().numpy()
    plt.title("pred (mean): "+ str(round(mean_age[0][0])) + "   true : "+ str(y_test[i]))
    # Compute the Gaussian distribution output
    # Ensure `ages` is converted properly and matches `d`'s device
    ages_tensor = torch.tensor(ages, dtype=torch.float32, device=d.loc.device)
    log_prob = d.log_prob(ages_tensor)
    # Convert to probability (exp of log_prob) and move to CPU for further processing
    ds = torch.exp(log_prob).detach().cpu().numpy()
    plt.subplot(1,2,2)
    plt.plot(ages, ds[0])
    plt.show()

#### Look at the predicted mean and the predicted sigma of the CPD on the testset


### Choose the one which are most sure

In [ ]:
batch_size = 100  # Adjust this value based on your memory capacity
uncertainty_flex = []
uncertainty_flex = model_flex.predict(X_test)
uncertainty_flex = np.array(uncertainty_flex)
#Sort the list and find the 5 highest values
indices = np.argsort(uncertainty_flex[:,0])[:5]
ages = np.arange(-1,8,0.1)
for i in indices:
    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)
    plt.imshow(X_test[i])
    d = output_to_gaussian_distribution(model_flex(X_test[i:i+1]))
    mean_age = d.mean.cpu().detach().numpy()
    plt.title("pred (mean): "+ str(round(mean_age[0][0],2)) + "   true : "+ str(y_test[i]))
    # Compute the Gaussian distribution output
    # Ensure `ages` is converted properly and matches `d`'s device
    ages_tensor = torch.tensor(ages, dtype=torch.float32, device=d.loc.device)
    log_prob = d.log_prob(ages_tensor)
    # Convert to probability (exp of log_prob) and move to CPU for further processing
    ds = torch.exp(log_prob).detach().cpu().numpy()
    plt.subplot(1,2,2)
    plt.plot(ages, ds[0])
    plt.show()

### NLL and MAE on the test set

In [ ]:
# Since the NLL is also the loss function, we can also use the evaluate function of keras. Which is much faster.
nll_flex = model_flex.evaluate(X_test,y_test,verbose=0)
print("NLL, MAE, and MSE for flexible sigma model: ", nll_flex)

Mean Absolute Error for the Null Model

In [ ]:
np.mean(np.abs((y_test - mu_naive)))

# Suggestion for further exercises (optional)

The gausian distribution is not the best distribution to model the age. For example the Log-Normal. This would give you a model that is not able to predict negative ages. And the result would look like.